In [1]:
import traceback

import polars as pl
import sys
import torch
import polars as pl
import numpy as np
from sklearn.linear_model import LinearRegression
from lifelines import WeibullFitter
from sklearn.model_selection import train_test_split
from torch import optim
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

from models.Titans import Model
from utils.custom_losses import CustomLoss
from utils.lstf_feature_maker.piecewise_linear_regression import PiecewiseLinearRegression
from utils.lstf_feature_maker.weibull import WeibullFeatureMaker

'''
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
https://developer.nvidia.com/cuda-12-8-0-download-archive
'''

MAC_DIR = '../data/'
WINDOW_DIR = 'C:/Users/USER/PycharmProjects/research/data/'

if sys.platform == 'win32':
    DIR = WINDOW_DIR
    print(torch.cuda.is_available())
    print(torch.cuda.device_count())
    print(torch.version.cuda)
    print(torch.__version__)
    print(torch.cuda.get_device_name(0))
    print(torch.__version__)
else:
    DIR = MAC_DIR

tb_bas_oper_part_mst = (pl.read_parquet(DIR + 'tb_bas_oper_part_mst.parquet')
                        .select(['OPER_PART_NO', 'OPER_PART_NM'])
                        .rename({'OPER_PART_NO': 'oper_part_no', 'OPER_PART_NM': 'oper_part_nm'}))
tb_dyn_fcst_demand = (pl.read_parquet(DIR + 'tb_dyn_fcst_dmnd.parquet')
                      .select(['PART_NO', 'DMND_QTY', 'DMND_DT', 'OPER_PART_NO'])
                      .rename({'PART_NO': 'part_no', 'OPER_PART_NO': 'oper_part_no', 'DMND_DT': 'demand_dt', 'DMND_QTY': 'demand_qty'})
                      .select(['part_no', 'oper_part_no', 'demand_dt', 'demand_qty']))
tb_dyn_fcst_demand_sellout = (pl.read_parquet(DIR + 'tb_dyn_fcst_dmnd_sellout.parquet')
                              .select(['PART_NO', 'DMND_QTY', 'DMND_DT', 'OPER_PART_NO'])
                              .rename({'PART_NO': 'part_no', 'OPER_PART_NO': 'oper_part_no', 'DMND_DT': 'demand_dt', 'DMND_QTY': 'demand_qty'})
                              .select(['part_no', 'oper_part_no', 'demand_dt', 'demand_qty']))

In [2]:
from utils.date_util import DateUtil

dyn_fcst_demand = tb_dyn_fcst_demand.with_columns([
    pl.col('demand_dt').cast(pl.Int64).map_elements(DateUtil.yyyymmdd_to_date, return_dtype = pl.Date)
])

dyn_demand_sellout = tb_dyn_fcst_demand_sellout.with_columns([
    pl.col('demand_dt').cast(pl.Int64).map_elements(DateUtil.yyyymmdd_to_date, return_dtype = pl.Date)
])

dyn_fcst_demand

part_no,oper_part_no,demand_dt,demand_qty
str,str,date,f64
"""T4240-71102BB""","""T4240-71102BB""",2018-01-01,3.0
"""T5210-34402""","""T5210-34402""",2018-01-01,1.0
"""T5210-30081""","""T5210-30081""",2018-01-01,1.0
"""T5210-65661""","""T5210-65661""",2018-01-01,1.0
"""T5210-66472""","""T5210-66472""",2018-01-01,1.0
…,…,…,…
"""U3215-52203""","""U3215-52203""",2024-02-05,30.0
"""T5710-69252""","""T5710-69252""",2024-02-05,2.0
"""DYD1-O07""","""DYD1-O07""",2024-02-05,4.0


In [3]:
dyn_fcst = (dyn_fcst_demand
                .join(tb_bas_oper_part_mst, on = 'oper_part_no', how = 'left')
                .select(['oper_part_no', 'oper_part_nm', 'demand_dt','demand_qty'])
                .sort(['oper_part_no', 'demand_dt'])
                .with_columns([
                    pl.col('demand_qty').cum_sum().over('oper_part_no').alias('cumsum_qty')
                ])
              )
dyn_demand = (dyn_demand_sellout.join(tb_bas_oper_part_mst, on = 'oper_part_no', how = 'left')
                    .select(['oper_part_no', 'oper_part_nm', 'demand_dt', 'demand_qty'])
                    .sort(['oper_part_no', 'demand_dt'])
                    .with_columns([
                        pl.col('demand_qty').cum_sum().over('oper_part_no').alias('cumsum_qty')
                 ])
               )

In [4]:
lookback_window = 40
horizon = 20
df_grouped = (dyn_demand
                .select(['oper_part_no', 'demand_dt', 'demand_qty', 'cumsum_qty'])
                .sort(['oper_part_no', 'demand_dt'])
                .with_columns(pl.col('demand_dt').map_elements(DateUtil.date_to_yyyymm, return_dtype = pl.Int64).alias('demand_dt'))
                .select('oper_part_no', 'demand_dt', 'demand_qty')
                .group_by(['oper_part_no', 'demand_dt'])
                .agg(pl.col('demand_qty').sum().alias('demand_qty'))
                .sort(['oper_part_no', 'demand_dt'])
              )

df_grouped

oper_part_no,demand_dt,demand_qty
str,i64,f64
"""0001-1001""",201803,5.0
"""0001-1001""",201811,7.0
"""0001-1001""",202002,120.0
"""0001-1001""",202003,2.0
"""0001-1001""",202305,2.0
…,…,…
"""ZZ90239""",202010,1.0
"""ZZ90239""",202111,1.0
"""ZZ90239""",202306,1.0


In [5]:
class MultiPartTimeSeriesDataset(Dataset):
    def __init__(self, df: pl.DataFrame, lookback: int, horizon: int):
        self.samples: list[tuple[np.ndarray, np.ndarray]] = []
        self.part_ids: list[str] = []

        grouped = df.sort(['oper_part_no', 'demand_dt']).partition_by('oper_part_no')

        for df in grouped:
            part_no = df['oper_part_no'][0]
            series = df['demand_qty'].to_numpy()

            for i in range(len(series) - lookback - horizon):
                x_seq = series[i:i + lookback]
                y_seq = series[i + lookback: i + lookback + horizon]
                self.samples.append((x_seq, y_seq))
                self.part_ids.append(part_no)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        x_seq, y_seq = self.samples[idx]
        return torch.tensor(x_seq, dtype = torch.float32).unsqueeze(-1),torch.tensor(y_seq, dtype = torch.float32), self.part_ids[idx]

In [8]:
def train_model(model, train_loader, val_loader, epochs=10, lr=1e-3, device='cuda'):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for x_batch, y_batch, _ in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            pred = model(x_batch)
            loss = loss_fn(pred, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_train_loss = total_loss / len(train_loader)

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x_val, y_val, _ in val_loader:
                x_val, y_val = x_val.to(device), y_val.to(device)
                pred = model(x_val)
                val_loss += loss_fn(pred, y_val).item()
        avg_val_loss = val_loss / len(val_loader)
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

In [9]:
from models.Titans import TestTimeMemoryManager

lookback = 48
horizon = 12

dataset = MultiPartTimeSeriesDataset(df_grouped, lookback, horizon)
train_size = int(len(dataset) * 0.8)
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size = 256, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 256)

model = Model(
        input_dim=1,
        d_model=128,
        n_layers=3,
        n_heads=4,
        d_ff=256,
        contextual_mem_size=64,
        persistent_mem_size=32,  # 메모리 용량 확장
        output_horizon=horizon
    )

    # ✅ 학습
device = "cuda" if torch.cuda.is_available() else "cpu"
train_model(model, train_loader, val_loader, epochs=10, lr=1e-3, device=device)

    # ✅ 특정 파트 예측 + Test-Time Memory Adaptation
test_part = "0001-1001"
test_series = (
    df_grouped.filter(pl.col("oper_part_no") == test_part)
        .sort("demand_dt")["demand_qty"].to_numpy()
)
x_new = torch.tensor(test_series[-(lookback+horizon):-horizon]).unsqueeze(0).unsqueeze(-1)
y_new = torch.tensor(test_series[-horizon:]).unsqueeze(0)

    # Memory Adaptation
ttm = TestTimeMemoryManager(model)
ttm.add_context(x_new)
loss_after_adapt = ttm.adapt(x_new, y_new, steps=3)
print(f"Adaptation Loss after TTM ({test_part}): {loss_after_adapt:.4f}")

    # ✅ 예측 결과
model.eval()
with torch.no_grad():
    pred = model(x_new)
print("Predicted:", pred.cpu().numpy().flatten())
print("Actual:", y_new.cpu().numpy().flatten())

Epoch 1/10 | Train Loss: 1658465.3041 | Val Loss: 3139851.7385
Epoch 2/10 | Train Loss: 1645888.6341 | Val Loss: 3123896.7920


KeyboardInterrupt: 

In [ ]:
df_grouped